## 5. Feature Importance and Permutation feature importance
In this notebook we are looking at the feature importances for the Random Forest Regressor.

In [31]:
# load modules
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from sklearn.model_selection import cross_val_predict
from modeling.features import get_feature_combinations
from sklearn.svm import SVR
from lightgbm import LGBMRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler


In [32]:
# loading the dataset
data = pd.read_csv('../data/GEFCom2014Data/Wind/clean_data.csv', 
                    parse_dates= ['TIMESTAMP'],
                    index_col= 'TIMESTAMP' )

In [33]:
# train-test-split and get features
data_train = data[:'2013-07-01 00:00:00']
data_test = data['2013-07-01 01:00:00']
feature_dict = get_feature_combinations()

### Feature importance

### Permutation feature importance